In [294]:
import numpy as np
import math

In [295]:
# algorithm 16
def CARATHEODORY(P, u):
    """
    author: Zhongxuan Liu
    :param P: a numpy array P of size n (points) times d (dimensions)
    :param u: weights function
    :return: a Caratheodory set (S, w)
    Computation time: O(n^2 d^2)
    """
    d = P.shape[1]
    while True:
        n = np.count_nonzero(u)
        u = u / np.sum(u)
        u_plus_idx = u > 0
        if n <= d + 1:
            return P, u

        A = P[u_plus_idx]
        P1 = np.outer(A[0], np.ones(A.shape[0] - 1))
        A = A[1:].T - P1

        _, _, V = np.linalg.svd(A)
        v = V[-1]
        v = np.insert(v, 0, -1 * sum(v))
        v_plus_idx = v > 0
        alpha = np.min(u[u_plus_idx][v_plus_idx] / v[v_plus_idx])

        w = np.zeros(P.shape[0])
        w_plus = u[u_plus_idx] - alpha * v
        w_plus[np.argmin(w_plus)] = 0.0
        w[u_plus_idx] = w_plus
        u = w

In [296]:
# algorithm 1
def FAST_CARATHEODORY_SET(P,P_index,u,k):
    # author: Shanshan Chen
    # P n*d array
    # u n array, weight vector for P
    d = P.shape[1]
    if(k < d+2):
        print("Warning: k < d+2 in FAST_CARATHEODORY_SET.")
        return P,P_index,u
    # 1.remove all points with zero weight
    weight_zero_index = u > 0
    P = P[weight_zero_index]
    P_index = P_index[weight_zero_index]
    # 2. P is already small
    n = P.shape[0]
    if(n <= d+1 or n < k):
        return P,P_index,u
    # 4. partition of P into k subsets
    t_lower = int(n/k)
    x = [i for i in range(0,k)]
    k_clusters_lable = np.array([],dtype = np.int32).reshape(0,1)
    for i in range(0,t_lower):
        k_clusters_lable = np.append(k_clusters_lable, x)
    rest = [i for i in range(0,n-t_lower*k)]
    k_clusters_lable = np.append(k_clusters_lable,rest)
    # 5.the weighted mean of P_k
    cls_w_means = np.array([],dtype = np.float64).reshape(0,d)
    cls_ws = np.array([],dtype = np.float64).reshape(0,1)
    for i in range(0,k):
        temp = k_clusters_lable == i
        P_i = P[temp]
        u_i = u[temp]
        denominator = np.sum(u_i)
        numerator =  np.dot(P_i.transpose(),u_i)
        cls_w_means = np.vstack((cls_w_means,numerator/denominator))
        cls_ws = np.append(cls_ws,denominator)
    # 8. algorithm 16
    selected_means, selected_ws = CARATHEODORY(cls_w_means,cls_ws) 
    selected_index = np.where(selected_ws > 0)[0]
    # 9. the union over all selected clusters
    C = np.array([],dtype = np.float64).reshape(0,d)
    C_index = np.array([],dtype = np.int32).reshape(0,1)
    w = np.array([],dtype = np.float64).reshape(0,1)
    for i in selected_index:
        temp = k_clusters_lable == i
        P_i = P[temp]
        index_i = P_index[temp]
        u_i = u[temp]
        w_i = selected_ws[i]*u_i/np.sum(u_i)
        C = np.vstack((C,P_i))
        C_index = np.append(C_index,index_i)
        w = np.append(w,w_i)
    # 12. recursive call
    C,C_index,w = FAST_CARATHEODORY_SET(C,C_index,w,k)
    return(C,C_index,w)

In [297]:
# algorithm 2
def CARATHEODORY_MATRIX(A,k):
    # author: Shanshan Chen
    n,d = A.shape
    if(k < d**2+2):
        print("Warning: k < d**2+2 in CARATHEODORY_MATRIX.")
        return A
    P = np.array([],dtype = np.float64).reshape(0,d**2)
    for i in range(0,n):
        a_i = A[i].reshape(-1,1)
        temp = np.dot(a_i,a_i.transpose()).reshape(1,-1)
        P = np.vstack((P,temp))
    P_index = np.arange(0,n)
    u = np.ones(n)/n
    C,C_index,w = FAST_CARATHEODORY_SET(P,P_index,u,k)
    S = np.array([],dtype = np.float64).reshape(0,d)
    for i in range(0,C.shape[0]):
        S_i = np.sqrt(n*w[i])*A[C_index[i]]
        S = np.vstack((S,S_i))
    return S

In [298]:
# algorithm 5
def LMS_CORESET(A, b, m, k):
    """
    author: Zhongxuan Liu
    modified by: Shanshan Chen
    This function computes a coreset for LMS solvers that use m-fold cross validation.
    The result satisfies ||Ax - b|| = ||Cx - y||.
    :param A: A matrix in R^{n * d}
    :param b: A vector in R^n
    :param m: A number of cross-validation folds
    :param k: An integer in range(1, n+1) for numerical accuracy/speed trade-off
    :return: A matrix C in R^{O(md^2)*d} and a vector y in R^n
    """
    d = A.shape[1]
    b = b.reshape(-1,1)
    A_prime = np.append(A, b,axis = 1)
    batch = A_prime.shape[0] // m

    S = CARATHEODORY_MATRIX(A_prime[:batch], k)
    S = S.T
    for i in range(1, m):
        Ai = A_prime[i * batch:(i + 1) * batch]
        Si = CARATHEODORY_MATRIX(Ai, k)
        S = np.hstack((S, Si.T))
    S = S.T
    C, y = S[:, :d], S[:, -1]
    return C, y

In [299]:
# test case for LMS_CORESET
A = np.arange(100).reshape(50,2)
x = np.array([1,2])
b = np.dot(A,x)
C,y = LMS_CORESET(A, b, 3, 12)

In [290]:
np.dot(C,x)

array([  2.753814  ,  73.34149978,  48.19445122,  92.65252447,
        19.63187925, 125.06372394,  26.62225252,  29.11352093,
        37.76192979, 110.82235785,  51.17624145, 237.84714216,
       202.7138877 ,  91.2608333 , 238.47287623,  98.07380037,
       173.3487394 , 287.06171029, 162.94276494, 159.12237645,
       375.36770485,  86.87236594, 131.24735599, 200.27707406,
       318.54352301, 273.95264171, 402.17739666, 284.57427061,
       290.6151416 , 465.49451446])

In [292]:
y

array([  2.753814  ,  73.34149978,  48.19445122,  92.65252447,
        19.63187925, 125.06372394,  26.62225252,  29.11352093,
        37.76192979, 110.82235785,  51.17624145, 237.84714216,
       202.7138877 ,  91.2608333 , 238.47287623,  98.07380037,
       173.3487394 , 287.06171029, 162.94276494, 159.12237645,
       375.36770485,  86.87236594, 131.24735599, 200.27707406,
       318.54352301, 273.95264171, 402.17739666, 284.57427061,
       290.6151416 , 465.49451446])

In [224]:
# test case for FAST_CARATHEODORY_SET
P = np.arange(24).reshape(12,2)
P_index = np.arange(12)
u = np.ones(12)/12
k = 5
C,C_index,w = FAST_CARATHEODORY_SET(P,P_index,u,k)

In [225]:
np.dot(P.transpose(),u)

array([11., 12.])

In [226]:
np.dot(C.transpose(),w)

array([11., 12.])

In [227]:
# test case for CARATHEODORY_MATRIX
A = np.arange(100).reshape(50,2)
k = 7
S = CARATHEODORY_MATRIX(A,k)

In [228]:
np.dot(A.transpose(),A)

array([[161700, 164150],
       [164150, 166650]])

In [229]:
np.dot(S.transpose(),S)

array([[161700., 164150.],
       [164150., 166650.]])